In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

'sudo'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'sudo'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.
'rm'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [2]:

# (로컬에서 실습할 경우) 파이토치 설치: conda install pytorch==1.12.0 -c pytorch 
# (로컬에서 실습할 경우) torchtext 설치: conda install -c pytorch torchtext 

# 데이터 다운로드 
import torchtext 
torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', 
                                  path=os.path.join(DATA_DIR, 'review.txt'))

ModuleNotFoundError: No module named 'torchtext'

In [ ]:

import pandas as pd
import numpy as np

import os, re
from tqdm import tqdm

# 경고문구 미표시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')

In [ ]:
DATA_DIR = "./data"
os.makedirs(DATA_DIR, exist_ok=True)

In [ ]:
import torchtext 
torchtext.utils.download_from_url(url='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', 
                                  path=os.path.join(DATA_DIR, 'review.txt'))

In [ ]:
# txt 파일을 판다스 데이터프레임으로 읽어오기
data = pd.read_csv(os.path.join(DATA_DIR, 'review.txt'), sep='\t')

# 데이터프레임의 크기
print(data.shape)

# 첫 5행을 출력
data.head()

In [ ]:
! pip install konlpy

In [ ]:
# 트위터 형태소 분석기(Okt)를 활용
from konlpy.utils import pprint
from konlpy.tag import Okt
okt = Okt()
print(okt)

In [ ]:
# 누락 데이터를 제거
review_data = data['document'].dropna().values

# 학습 속도를 고려하여, 1000개의 샘플을 선택하여 추출
review_data = review_data[:1000]

# 배열의 크기
print(review_data.shape)

# 첫 번째 데이터
print(review_data[0])

In [ ]:
# 세 글자 이상의 명사를 사용 (두 글자 이하의 단어는 제거)
cleaned_review_data = []

for review in tqdm(review_data):
    tokens = okt.nouns(review)
    cleaned_tokens = []
    for word in tokens:
        if len(word) > 2:
            cleaned_tokens.append(word)
        else:
            pass
    cleaned_review = " ".join(cleaned_tokens)
    cleaned_review_data.append(cleaned_review)

print(len(cleaned_review_data))
print(cleaned_review_data[0])

In [ ]:
# 사이킷런 패키지 활용 
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 변환기 객체를 생성
tfid = TfidfVectorizer()

# TF-IDF 변환기에 데이터를 입력하여 변환
review_tfid = tfid.fit_transform(cleaned_review_data)

# 배열의 크기
print(review_tfid.shape)

# 첫 번째 데이터
print(review_tfid[0])

In [ ]:
# 단어 사전 확인 (딕셔너리 형태)
vocab = tfid.vocabulary_

# 단어 사전의 크기
print(len(vocab))

# 단어 사전 출력 (앞에서 5개의 단어만 출력)
print({ k:v for i, (k, v) in enumerate(vocab.items()) if i < 5 })

In [ ]:
# 단어들의 사전 인덱스를 이용하여 원래 단어를 검색하는 매핑 딕셔너리
index_to_word = { v:k for k, v in vocab.items() } 

# 앞에서 5개의 단어를 출력
print({  k:v for i, (k, v) in enumerate(index_to_word.items()) if i < 5 })

In [ ]:
# 첫 번째 리뷰를 구성하는 단어들의 사전 인덱스를 이용하여 원래 단어를 복원 (순서 복원 X)
original_text = " ".join([index_to_word[word_idx] for word_idx in review_tfid[0].indices])
original_text

In [ ]:
# 사이킷런 패키지 활용
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델링 객체를 생성 (토픽 갯수를 2로 설정: 긍정/부정)
lda = LatentDirichletAllocation(n_components=2)  

# TF-IDF 벡터를 입력하여 모델 학습 
lda.fit(review_tfid)

In [ ]:
# 토픽 모델링 결과를 담고 있는 배열의 형태 : (2개의 토픽, 2157개의 단어)
print(lda.components_.shape)

In [ ]:
# 2157개의 단어 중에서, 토픽 별로 가장 중요도가 높은 단어를 10개씩 출력

for idx, topic in enumerate(lda.components_):
    print(f"토픽 유형 {idx+1}:", [(index_to_word[i], topic[i].round(3)) for i in topic.argsort()[:-11:-1]])

In [ ]:
# pyLDAvis 설치
!pip install pyLDAvis

In [ ]:
# LDA 토픽 모델링 결과를 시각화
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
visualization = pyLDAvis.sklearn.prepare(lda, review_tfid, tfid)
pyLDAvis.display(visualization)

In [ ]:
labels = data['label'].iloc[:1000].values
print(labels.shape)
print(labels[:5])

In [ ]:
# 사이킷런 패키지 활용
from sklearn.linear_model import LogisticRegression

# 로지스틱 분류 모델링 객체를 생성 
lr = LogisticRegression()

# TF-IDF 벡터를 입력하여 모델 학습 
lr.fit(review_tfid, labels)

In [ ]:
# 첫 번째 샘플을 이용하여 모델 예측
pred = lr.predict(review_tfid[0])
print(pred)

In [ ]:
# 판다스 데이터프레임로 읽어오기
test = pd.read_csv('naver_review.csv')

test.head()

In [ ]:

# 판다스 데이터프레임로 읽어오기
test = pd.read_csv('naver_review.csv')

test.head()


In [ ]:
# 첫 번째 리뷰를 선택
test_sample = test['review'][0]
test_sample

In [ ]:
# 한글을 제외하고 전부 제거
test_sample = re.sub(r"[^가-힣]", "", test_sample)
test_sample

In [ ]:
# 세 글자 이상의 명사만을 추출

tokens = okt.nouns(test_sample)
cleaned_tokens = []
for word in tokens:
    if len(word) > 2:
        cleaned_tokens.append(word)
    else:
        pass
cleaned_review = " ".join(cleaned_tokens)

cleaned_review

In [ ]:
# TF-IDF 변환기에 데이터를 입력하여 변환
test_review_tfid = tfid.transform([cleaned_review])

# 배열을 출력
print(test_review_tfid)

In [ ]:
# 로지스틱 회귀 모델을 활용하여 분류 예측
test_pred = lr.predict(test_review_tfid)[0]
print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if test_pred > 0 else "부정"))

In [ ]:
result = test['review'][:]
test = result.to_frame()
test

In [ ]:
def f(review):


  result1 = re.sub(r"[^가-힣]", "", str(result))
  return result1

result['fin_review'] = test['review'].apply(f)
result.to_frame()

In [ ]:
tokens = okt.nouns(result)
cleaned_result = []
for word in tokens:
    if len(word) > 2:
        cleaned_result.append(word)
    else:
        pass
cleaned_review_fin = " ".join(cleaned_result)

cleaned_review_fin

In [ ]:
result_review_tfid = tfid.transform([cleaned_review_fin])

for i in range(10):
  result_pred = lr.predict(result_review_tfid)[i]
  print("분석 결과 {}적인 리뷰로 예측됩니다. ".format("긍정" if test_pred > 0 else "부정"))